# Activity 3: Gaussian Elimination II: Solving general systems

In this activity, we'll expand on our work from last time to give a more flexible form of our Gaussian Elimination routines and solve linear systems in greater generality.

In [2]:
import numpy as np

## Reminder: `while` loops
You may have gotten through the last activity only using `for` loops. In this activity, you may find it more convenient to use `while` loops. These have the advantage of terminating immediately when a specified condition is met and a possible disadvantage of requiring you to increment your parameters manually.

As a demonstration, below is a function which adds up half of all the entries of a list of integers before the first odd entry, printing the running total each time.

In [3]:
def add_up_starting_evens(l):
    tot=0 #initialize sum
    even_found=False #initialize Boolean which will eventually terminate our while loop
    j=0 #initialize our incremental parameter
    while even_found==False:
        if l[j] % 2 == 0: #if the j'th entry is even
            tot=tot+l[j]//2 #add half the j'th entry to tot
            print(tot,",", end =" ") #print our running total
            j+=1 #increment j by 1-- this is equivalent to "j=j+1"
        else: #if j'th entry is odd
            even_found=True
    return tot

In [4]:
add_up_starting_evens([2,4,8,26,15,98,96,90])

1 , 3 , 7 , 20 , 

20

In [5]:
add_up_starting_evens([1,4,4,4,4,4,4,4,4,4,4,4,4])

0

In [6]:
add_up_starting_evens([4,4,4,4,4,4,4,4,4,4,4,4,1])

2 , 4 , 6 , 8 , 10 , 12 , 14 , 16 , 18 , 20 , 22 , 24 , 

24

Your turn: Use a `while` loop to write a function which takes a positive integer `n` for input and performs the following repeated Collatz process:
- if  `n` is even, replace `n` with `n//2` and `print` it
- if `n` is odd and not equal to 1, replace `n` with `3*n+1` and `print` it
- if `n` is 1, terminate the program and return the number of steps it took to get there (recorded as `nsteps`).
It's a somewhat famous unsolved problem to show that this process terminates for any positive integer as input!

To make sure your output is readable, use the following `print` command: `print(n,",", end =" ")`. This makes it so that each `print` is not on its own line.

You might want to take a moment to think about how to do this with `while` loops. One idea is to use a Boolean like the example above, but you can also quantify a `while` loop with any equation or inequality--is there something involving `n` that will work?

In [7]:
def my_collatz(n):
    nsteps = 0
    while n != 1:
        if n % 2 == 0:
            n = n // 2
        else:
            n = 3 * n + 1
        print(n, ",", end=" ")
        nsteps += 1
    return nsteps

In [8]:
#Testing
my_collatz(99)
#Desired output:
#298 , 149 , 448 , 224 , 112 , 56 , 28 , 14 , 7 , 22 , 11 , 34 , 17 , 52 , 26 , 13 , 40 , 20 , 10 , 5 , 16 , 8 , 4 , 2 , 1 ,

# 25

298 , 149 , 448 , 224 , 112 , 56 , 28 , 14 , 7 , 22 , 11 , 34 , 17 , 52 , 26 , 13 , 40 , 20 , 10 , 5 , 16 , 8 , 4 , 2 , 1 , 

25

## Exercise 1: The Permuted LU Factorization
In the previous activity, we saw how to compute an $LU$ factorization of $A$ when $A$ is *regular*.

Now, we'll promote that to all nonsingular matrices with the $PA=LU$ factorization.

First, we may want some way of generating elementary permutation matrices. One way you might do that is by using slicing over a list of rows. That is, (for instance) `A[[1,3]]` would give the second and fourth rows of `A`, and `A[[3,1]]` gives the fourth and second rows (in that order), each as a view (rather than a copy).

Now, use that (or any other method) to give a function generating the `n`-by-`n` permutation matrix which swaps rows `i` and `j`. As another reminder, `np.identity(n)` gives the `n`-by-`n` identity matrix.

In [9]:
def perm_matrix():
    E = np.identity(n)
    if i != j:
        E[[i, j]] = E[[j, i]]
    return E

Now, let's build our $PA=LU$ function, starting with your `my_LU` from activity 2. Your function should take a matrix `A` as input and give a tuple of matrices `(P,L,U)` as output where $LU$ form an LU factorization of the matrix $PA$.
The process for computing the factorization is on pages 28-29 of Olver-Shakiban.
You may do this however you wish, but the following suggestions may be useful:


- You may want to make use of *subroutines*, that is, functions which exist only in service of a bigger function. One which may be useful is a pivoting routine -- let's call it `swapper`. This should implement the pivoting process described in the paragraph beneath Theorem 1.11 in Olver-Shakiban.

  - In particular, `swapper` should take as input your partially computed `P`, `L` and `U` as well as rows `i` and `k` to swap (where `i<j`). The output should be a now-pivoted `P`, `L`, and `U`.
   - Implementing pivoting for `P` and `U` is as simple as multiplying by the correct elementary permutation matrix to swap rows `i` and `k,`. Handling `L` is more delicate.
   <details>
    <summary> <b> Hint:</b> (Click here to expand) </summary>
    Letting $P_{ik}$ be the elementary matrix in question, the pivoted form of $L$ is equal to $P_{ik}(L-I)+I$ where $I$ is the $n\times n $ identity matrix. Can you use that in your implementation? (This is not the only way to accomplish this)
    </details>
- With `swapper` in hand, we now modify our `my_LU` function to give a $PA=LU$ factorization.
  - After exercise 1 of activity 2, your `my_LU` should `raise` an `Exception` when a diagonal entry zero. Now instead, when looking for a pivot in row `i` and finding `U[i][i]=0`, you should look to pivot by checking for nonzero entries below your diagonal.
  <details>
    <summary> <b> Hint:</b> (Click here to expand) </summary>
    
    One way to do this is with a while loop. Let ``k`` be the parameter determining which row below row ``i`` we are checking. Initialize a boolean (let's call it ``swapped``) with ``swapped=False`` and then set up a while loop over the condition that ``k<n`` (so that we don't attempt to check beneath the bottom row) and  ``swapped=False`` (so that we don't redundantly swap multiple times. Once you find a nonzero entry, employ swapper to implement the swap.
    </details>After

In [10]:
def elem_matrix(n, i, k):
    E = np.identity(n)
    if i != k:
        E[[i, k]] = E[[k, i]]
    return E

def swapper(P, L, U, i, k):
    n = U.shape[0]
    E = elem_matrix(n, i, k)
    U = E @ U
    P = E @ P
    if k > 0:
        L[[i, k], :k] = L[[k, i], :k]
    return P, L, U

def my_PLU(A):
    A = np.array(A, dtype=float)
    n = A.shape[0]
    P = np.identity(n)
    L = np.identity(n)
    U = A.copy()
    for k in range(n - 1):
        if U[k, k] == 0:
            p = None
            for r in range(k + 1, n):
                if U[r, k] != 0:
                    p = r
                    break
            if p is not None:
                P, L, U = swapper(P, L, U, p, k)
        for i in range(k + 1, n):
            L[i, k] = U[i, k] / U[k, k]
            U[i, k:] -= L[i, k] * U[k, k:]
            U[i, k] = 0.0
    return P, L, U


In [11]:
#testing
A1=np.array([[ 1.,  2., -1.,  0.],
            [-1., -2., 11., 11.],
            [ 1.,  3.,  2.,  1.],
            [ 2.,  5., -4., -5.]],"float64")
(P1,L1,U1)=my_PLU(A1)
print("P1=\n",P1,"\nL1=\n",L1,"\nU1=\n",U1,"\nP1A1=L1U1?\n", np.array_equal(np.dot(P1,A1),np.dot(L1,U1)))
#Possible desired output:
#P=
# [[1. 0. 0. 0.]
# [0. 0. 1. 0.]
# [0. 1. 0. 0.]
# [0. 0. 0. 1.]]
#L=
# [[ 1.   0.   0.   0. ]
# [ 1.   1.   0.   0. ]
# [-1.   0.   1.   0. ]
# [ 2.   1.  -0.5  1. ]]
#U=
# [[ 1.   2.  -1.   0. ]
# [ 0.   1.   3.   1. ]
# [ 0.   0.  10.  11. ]
# [ 0.   0.   0.  -0.5]]
#PA=LU?
# True

P1=
 [[1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]] 
L1=
 [[ 1.   0.   0.   0. ]
 [ 1.   1.   0.   0. ]
 [-1.   0.   1.   0. ]
 [ 2.   1.  -0.5  1. ]] 
U1=
 [[ 1.   2.  -1.   0. ]
 [ 0.   1.   3.   1. ]
 [ 0.   0.  10.  11. ]
 [ 0.   0.   0.  -0.5]] 
P1A1=L1U1?
 True


___

## Exercise 2: General systems
Now, we'll expand our `my_PLU` to handle general homogeneous systems. That is, so far we have assumed square input matrix `A`. Now, expand `my_PLU` to give a $PA=LU$ factorization for an $n\times m$ matrix `A` which is not assumed to be full rank (here, U is assumed only to be REF) <details>
    <summary> <b> Hints:</b> (Click here to expand) </summary>

- Recall the general idea of finding the REF form of a matrix: you work down and to the right finding pivots and clearing out beneath them. You know you are done when either you have found and cleared out below $n-1$ pivots or when you have exhausted every column.
- You will likely want to use parameters `i` and `j` to record the row and column of the entry you are checking as a pivot.
- For your main loop (finding pivots--very likely a `while` loop), what stop condition implements the stop condition described above?
       <details>
           <summary> <b> Hint: </b> (click here to expand) </summary>      
           `while i<n-1 and j<m`
    </details>
-   Your subloop for implementing row-swaps can likely go unchanged  
    
    </details>

In [12]:
def my_REF(A):
    A = np.array(A, dtype=float)
    n, m = A.shape
    P = np.identity(n)
    L = np.identity(n)
    U = A.copy()

    for k in range(min(n, m)):
        if abs(U[k, k]) < 1e-12:
            p = None
            for r in range(k + 1, n):
                if abs(U[r, k]) > 1e-12:
                    p = r
                    break
            if p is not None:
                U[[k, p], :] = U[[p, k], :]
                P[[k, p], :] = P[[p, k], :]
                if k > 0:
                    L[[k, p], :k] = L[[p, k], :k]
            else:
                continue
        for i in range(k + 1, n):
            if abs(U[i, k]) > 1e-12:
                L[i, k] = U[i, k] / U[k, k]
                U[i, k:] -= L[i, k] * U[k, k:]
                U[i, k] = 0.0

    return P, L, U

In [13]:
#testing
A2=np.array([[  8.,  12.,   -6.,  92., -20.],
            [  2.,   3.,   5.,  14.,  11.],
            [ -4.,  -6.,   3., -54.,  22.]],"float64")
(P2,L2,U2)=my_REF(A2)
print("P2=\n",P2,"\nL2=\n",L2,"\nU2=\n",U2,"\nP2A2=L2U2?\n", np.array_equal(np.dot(P2,A2),np.dot(L2,U2)))
#Desired output:
#P2=
# [[1. 0. 0.]
# [0. 1. 0.]
# [0. 0. 1.]]
#L2=
# [[ 1.    0.    0.  ]
# [ 0.25  1.    0.  ]
# [-0.5   0.    1.  ]]
#U2=
# [[  8.   12.   -6.   92.  -20. ]
# [  0.    0.    6.5  -9.   16. ]
# [  0.    0.    0.   -8.   12. ]]
#P2A2=L2U2?
# True

P2=
 [[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]] 
L2=
 [[ 1.    0.    0.  ]
 [ 0.25  1.    0.  ]
 [-0.5   0.    1.  ]] 
U2=
 [[  8.   12.   -6.   92.  -20. ]
 [  0.    0.    6.5  -9.   16. ]
 [  0.    0.    0.   -8.   12. ]] 
P2A2=L2U2?
 True


In [14]:
#testing
A3=np.array([[1,2,2,-1,0,0],
            [-1,-2,-2,1,0,0],
            [1,2,3,2,3,2],
            [-2,-4,-7,-12,-14,-12]],"float64")
(P3,L3,U3)=my_REF(A3)
print("P3=\n",P3,"\nL3=\n",L3,"\nU3=\n",U3,"\nP3A3=L3U3?\n", np.array_equal(np.dot(P3,A3),np.dot(L3,U3)))
#Possible Desired Output:
#P3=
# [[1. 0. 0. 0.]
# [0. 0. 1. 0.]
# [0. 0. 0. 1.]
# [0. 1. 0. 0.]]
#L3=
# [[ 1.  0.  0.  0.]
# [ 1.  1.  0.  0.]
# [-2. -3.  1.  0.]
# [-1.  0.  0.  1.]]
#U3=
# [[ 1.  2.  2. -1.  0.  0.]
# [ 0.  0.  1.  3.  3.  2.]Now
# [ 0.  0.  0. -5. -5. -6.]
# [ 0.  0.  0.  0.  0.  0.]]
#P3A3=L3U3?
# True



P3=
 [[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]] 
L3=
 [[ 1.  0.  0.  0.]
 [-1.  1.  0.  0.]
 [ 1.  0.  1.  0.]
 [-2.  0. -3.  1.]] 
U3=
 [[ 1.  2.  2. -1.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  3.  3.  2.]
 [ 0.  0.  0. -5. -5. -6.]] 
P3A3=L3U3?
 True


___

## Exercise 3: RREF and Homogeneous systems
**Definition:** A matrix $V$ is said to be in *reduced row echelon form* (RREF) if (i) $V$ is in row echelon form and (ii) each pivot is equal to 1 and is the only nonzero entry of its row.

Any matrix $A$ is equivalent to a RREF matrix $V$. To see this, let $U$ be the REF form of $A$. Then, working from the bottom row up, if row $i$ is nonzero with pivot $u_{ij}$, multiply that row by $\frac{1}{u_{ij}}$ and add appropriate multiples of row $i$ to the rows above it to clear out those entries.  

For example, for the REF matrix
$$U=\begin{pmatrix}
2 & 4 & 5 & 14 & 11 \\
0 & 0 & -2 & 4 & -6 \\
0 & 0 & 0 & 0 & -10 \\
\end{pmatrix},$$
its RREF form would be
$$V=\begin{pmatrix}
1 & 2 & 0 & 12 & 0 \\
0 & 0 & 1 & -2 & 0 \\
0 & 0 & 0 & 0 & 1 \\
\end{pmatrix}.$$


<b> (a) </b> We've now worked all the way through finding REF forms of very general systems. Now, write a function `RREFify` which takes as input a REF matrix `U` and gives as output an RREF matrix `V`. Then, write a wrapper `my_RREF` which takes a general matrix `A` and (using `my_REF` and `my_RREFify`) gives the RREF form `V`.

In [15]:
def RREFify(U):
    U = np.array(U, dtype=float)
    V = U.copy()
    n, m = V.shape
    for i in range(n - 1, -1, -1):
        nz = np.where(np.abs(V[i]) > 1e-12)[0]
        if nz.size == 0:
            continue
        j = nz[0]
        V[i, :] /= V[i, j]
        for r in range(i):
            V[r, :] -= V[r, j] * V[i, :]
    return V

def my_RREF(A):
    P, L, U = my_REF(A)
    return RREFify(U)

In [16]:
#testing
U=np.array([[2,4,5,14,11],
            [0,0,-2,4,-6],
            [0,0,0,0,-10]],"float64")
RREFify(U)
#Desired Output:
#array([[ 1.,  2.,  0., 12.,  0.],
#       [-0., -0.,  1., -2., -0.],
#       [-0., -0., -0., -0.,  1.]])

array([[ 1.,  2.,  0., 12.,  0.],
       [-0., -0.,  1., -2., -0.],
       [-0., -0., -0., -0.,  1.]])

___

<b> (b) Challenge! </b> Use this to give the space of solutions to the homogeneous system of equations represented by a matrix `A`. Present your answer as an array `B`, the columns of which generate the solution space.

<details>
    <summary>
        <b>Hint</b>: (click here to open) </summary>
    
- You might find it handy to have some way of generating a list of pivots/free variables of an RREF matrix. This could be accomplished with a subroutine, or by simply modifying ```RREFify```.
    </details>

In [17]:
def my_kernel(A):
    V = my_RREF(A).astype(float)
    r, c = V.shape
    eps = 1e-12
    piv = []
    for i in range(r):
        nz = np.flatnonzero(np.abs(V[i]) > eps)
        if nz.size: piv.append(nz[0])
    free = [j for j in range(c) if j not in set(piv)]
    B = []
    for j in free:
        x = np.zeros(c)
        x[j] = 1.0
        for i in range(r):
            nz = np.flatnonzero(np.abs(V[i]) > eps)
            if nz.size: x[nz[0]] = -V[i, j]
        B.append(x)
    return np.column_stack(B) if B else np.zeros((c, 0))

In [18]:
#Testing
A4=np.array([[ 1.,  2.,  3.,  4.,  5.],
             [ 2.,  4., 10., 14., 18.],
             [ 9., 18., 27., 36., 45.]],"float64")
B4=my_kernel(A4)
print("B4=\n",B4)
print("A4B4=\n",np.dot(A4,B4)) #check we actually get zero

B4=
 [[-2.   0.5  1. ]
 [ 1.   0.   0. ]
 [-0.  -1.5 -2. ]
 [ 0.   1.   0. ]
 [ 0.   0.   1. ]]
A4B4=
 [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
